In [ ]:
!pip install langchain openai chromadb unstructured pdfminer.six tiktoken

In [45]:
# !pip install unstructured[local-inference]
!pip install streamlit 

  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ------------ --------------------------- 3.1/10.1 MB 15.6 MB/s eta 0:00:01
   ------------------------- -------------- 6.3/10.1 MB 15.2 MB/s eta 0:00:01
   ------------------------------------ --- 9.2/10.1 MB 15.0 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 14.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 10.9 MB/s eta 0:00:00
Using cached GitPython-3.1.44-py3-none-any.whl (207 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ------------------ --------------------- 3.1/6.9 MB 14.9 MB/s eta 0:00:01
   -------------------

In [47]:
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# import pi_heif

loader = UnstructuredPDFLoader("reliance.pdf")
docs = loader.load()

# Split long text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)


Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P1' is an invalid float value
Cannot set gray stroke color because /'P2' is an invalid float value
Cannot set gray stroke color because /'P3' is an invalid float value
Cannot set gray stroke color because /'P4' is an invalid float value
Cannot set gray stroke color because /'P5' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P2' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value


In [57]:
loader

In [56]:
len(docs[0].page_content)

83267

In [29]:
documents[4]

Document(metadata={'source': 'reliance.pdf'}, page_content='7.3%\n\nEARNINGS PER SHARE (C) ₹102.9\n\n5.0%\n\nIndia’s foremost retailer with industry-leading reach, revenue, and profitability operating an integrated network of stores and digital commerce platforms.\n\nConsumption Baskets\n\nConsumer Electronics, Grocery, Fashion and Lifestyle and Connectivity\n\nREVENUE ₹3,06,848 Crore US$ 36.8 Billion EBITDA ₹23,082 Crore US$ 2.8 Billion\n\nFY 2023-24\n\nFY 2022-23\n\nFY 2021-22\n\n10,00,122 9,74,864 7,88,743\n\nStrong revenue growth, supported by continued growth momentum in consumer businesses and upstream business.\n\nFY 2023-24\n\nFY 2022-23\n\nFY 2021-22\n\nRecord net profit, led by strong operational performance across all businesses.\n\n79,020 73,670 66,184\n\nFY 2023-24\n\nFY 2022-23\n\nFY 2021-22\n\nStrong operating performance, with all businesses contributing to earnings growth.\n\n102.9 98.0 89.5\n\nDigital Services')

In [27]:
documents[0].page_content


'RETAIL\n\nENTERTAINMENT\n\nDIGITAL SERVICES\n\nReliance for\n\nENERGY\n\nMATERIALS\n\nIntegrated Annual Report 2023-24\n\nEMPOWERMENT\n\nSPORTS\n\nEDUCATION\n\nReliance Industries Limited (RIL) is a Fortune Global 500 company and the largest private sector company in India. The growth of Reliance mirrors the relentless spirit of dynamism and hope that defines India. It is this spirit that Reliance is committed to foster, and it is articulated in our timeless expression of intent, ‘Growth is Life’.\n\nAbout this Report\n\nThe Reliance Integrated Annual Report has been prepared in alignment with the Integrated Reporting <IR> Framework. In preparation for the Report, GRI Standards, National Guidelines for Responsible Business Conduct (NGRBC), United Nations Sustainable Development Goals (UN SDGs) and 13 other frameworks were referenced. The Report outlines RIL’s commitment to stakeholder value creation and defines the actions taken and outcomes achieved for it’s stakeholders.'

In [ ]:
len(documents[0].page_content)


In [58]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding = OpenAIEmbeddings()

# Create a vectorstore
vectorstore = Chroma.from_documents(documents, embedding)


In [76]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


system_prompt = (
    "Use the given pdf only to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use four sentence maximum and keep the answer concise. "
    "Context: {context}"
)



prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

question_answer_chain = create_stuff_documents_chain(llm, prompt)
qa_chain = create_retrieval_chain(vectorstore.as_retriever(), question_answer_chain)


In [77]:
query = "What are the main findings discussed in the PDF?"
response = qa_chain.invoke({"input": query})

print(response)

{'input': 'What are the main findings discussed in the PDF?', 'context': [Document(metadata={'source': 'reliance.pdf'}, page_content='Note: For more details on Capitals, please refer Page 38.\n\n10\n\n11\n\nManagement Discussion and Analysis\n\nFinancial Performance and Review\n\nSrikanth Venkatachari\n\nSoumyo Dutta\n\nAnshuman Thakur\n\nDinesh Taluja\n\nSaurabh Sancheti\n\nC. S. Borar\n\nRaj Mullick\n\nDespite global headwinds, India’s economic performance was surprisingly robust, catalysed by strong\n\ndomestic consumption and a pick-up in investment.\n\nSumit Mantri\n\nFinancial Performance and Review\n\nPage 13\n\nBusiness Overview\n\nPage 15\n\n15 Retail 15 Industry Overview 16 Business Performance 17 SCOT Analysis 17 Outlook\n\n24 Oil to\n\nChemicals Industry Overview 24 25 Business Performance 26 SCOT Analysis 26 Outlook'), Document(metadata={'source': 'reliance.pdf'}, page_content='Note: For more details on Capitals, please refer Page 38.\n\n10\n\n11\n\nManagement Discussion a

In [78]:
response['answer']

"The main findings discussed in the PDF are related to India's economic performance being robust due to strong domestic consumption and increased investment despite global headwinds. The report also mentions forward-looking statements regarding the company's growth strategy, product development, market position, and expenditures. Additionally, it highlights the significant volatility in global financial markets in FY 2023-24, with unpredictable shifts in sentiments from growth concerns to inflation worries, particularly in the US."

In [79]:
query = "explain SCOT analysis discussed in the PDF"
response = qa_chain.invoke({"input":query})

response['answer']

"The SCOT analysis in the PDF refers to an analysis of the company's Strengths, Challenges, Opportunities, and Threats. It highlights factors such as limited presence in end-user markets, leveraging technology for digitized experiences, meeting sustainability mandates, global overcapacity in certain products, energy market volatility, and exploiting emerging trends like EV charging networks. The analysis provides insights into how these factors can impact the company's market penetration, operational efficiency, profitability, competitiveness, and overall market position."

In [ ]:
query = "who will be next prime minister of india"  #just to validate system prompt
response = qa_chain.invoke({"input":query})

response['answer']

"I don't know."